In [1]:
import tensorflow as tf
import keras
from keras import layers, optimizers
from keras.metrics import Precision
from keras.layers import Conv2D, Input,MaxPool2D, GlobalAveragePooling2D, Dense
from sklearn.model_selection import train_test_split
import numpy as np


2025-12-11 22:26:54.595912: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr

In [2]:
DATA_PATH = "../data/anklealign/anklealign"

In [6]:
dataset = tf.data.Dataset.load(DATA_PATH+"/dataset")

In [8]:
import tensorflow as tf

classes = ['1_Pronacio', '2_Neutralis', '3_Szupinacio']
class_to_index = {c: i for i, c in enumerate(classes)}
num_classes = len(classes)

# string -> index TF-ben
lookup_table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(classes),
        values=tf.constant(list(range(num_classes)), dtype=tf.int32),
    ),
    default_value=-1
)

def encode_one_hot_tf(x, y):
    y_idx = lookup_table.lookup(y)            # tf.string -> int
    y_one_hot = tf.one_hot(y_idx, depth=num_classes)
    x.set_shape([256,256,3])                  # fix input shape
    return x, y_one_hot

dataset = dataset.map(encode_one_hot_tf)


In [9]:
dataset_size = len(dataset)
dataset = dataset.shuffle(buffer_size=dataset_size, seed=42)

train_size = int(0.64 * dataset_size)
val_size = int(0.16 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)


In [12]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Feltételezzük: dataset = (x, y), y one-hot
# Cél: minden elemre ugyanaz a predikció [0,1,0] -> 'Neutralis'

baseline_one_hot = np.array([0,1,0])
num_classes = 3

y_true_list = []
y_pred_list = []

for x, y in dataset:
    y_true_list.append(y.numpy())               # valós címke
    y_pred_list.append(baseline_one_hot)       # baseline predikció

# Konvertálás numpy tömbbé
y_true = np.array(y_true_list)
y_pred = np.array(y_pred_list)

# One-hot -> class index
y_true_idx = np.argmax(y_true, axis=1)
y_pred_idx = np.argmax(y_pred, axis=1)

# Metrikák
acc = accuracy_score(y_true_idx, y_pred_idx)
cm = confusion_matrix(y_true_idx, y_pred_idx)
report = classification_report(y_true_idx, y_pred_idx, target_names=['Pronacio','Neutralis','Szupinacio'])

print("Baseline Accuracy:", acc)
print("Confusion Matrix:\n", cm)
print(report)


Baseline Accuracy: 0.4117647058823529
Confusion Matrix:
 [[  0 146   0]
 [  0 140   0]
 [  0  54   0]]
              precision    recall  f1-score   support

    Pronacio       0.00      0.00      0.00       146
   Neutralis       0.41      1.00      0.58       140
  Szupinacio       0.00      0.00      0.00        54

    accuracy                           0.41       340
   macro avg       0.14      0.33      0.19       340
weighted avg       0.17      0.41      0.24       340



2025-12-11 22:33:39.246462: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio

In [ ]:
def creatingCNNModel():
    pass

In [ ]:
x = np.random.rand(1, 10, 10, 128)
Conv2D(32,3, activation='relu')(x).shape

In [10]:
model = keras.Sequential(
    [      
        Input((256,256,3)),
        Conv2D(4, (3,3), activation='relu'),
        MaxPool2D((2,2)),
        Conv2D(8, (3,3), activation='relu'),
        GlobalAveragePooling2D(),
        Dense(3)
    ]
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 4)    │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 8)    │           296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 8)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 435 (1.70 KB)

 Trainable params: 435 (1.70 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
train_dataset = train_dataset.batch(32, drop_remainder=True)
val_dataset   = val_dataset.batch(32, drop_remainder=True)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 576ms/step - accuracy: 0.4167 - loss: 6.9731 - val_accuracy: 0.4062 - val_loss: 6.7862
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 561ms/step - accuracy: 0.4427 - loss: 7.3496 - val_accuracy: 0.4375 - val_loss: 7.2440
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 527ms/step - accuracy: 0.4427 - loss: 7.2734 - val_accuracy: 0.4062 - val_loss: 6.6018
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 598ms/step - accuracy: 0.4688 - loss: 7.9711 - val_accuracy: 0.5938 - val_loss: 5.2328
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 539ms/step - accuracy: 0.4635 - loss: 6.3437 - val_accuracy: 0.5000 - val_loss: 5.7630
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 578ms/step - accuracy: 0.4323 - loss: 7.1782 - val_accuracy: 0.6250 - val_loss: 5.0698
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 575ms/step - accuracy: 0.4219 - loss: 6.7428 - val_accuracy: 0.3438 - val_loss: 7.5554
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 555ms/step - accuracy: 0.4323 - loss: 8.3948 - val_accuracy: 0.4688 - val_loss: